In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
os.chdir('/mnt/Storage/home/wangyiman/CHMsInOtherContexts/CellStateTransition/DataCollection')

In [10]:
data_geo_dict = {}
# for tissue in ['heart', 'liver'] :
for tissue in ['heart', 'liver'] :
    data_geo_dict[tissue] = pd.read_csv(f'DataCollection_add_{tissue}_RNA.csv', header = 0, sep = ",", usecols = ['Stage', 'Data', 'GSMID', 'GSEID'])
    data_geo_dict[tissue] = data_geo_dict[tissue].loc[data_geo_dict[tissue]['Data'] == 'RNA-seq']

data_geo_dict[tissue].head()

,Stage,Data,GSMID,GSEID
58,E11.5 Liver,RNA-seq,GSM2071370,GSE78357
59,E11.5 Liver,RNA-seq,GSM2071371,GSE78357
60,E12.5 Liver,RNA-seq,GSM2192083,GSE82853
61,E12.5 Liver,RNA-seq,GSM2192084,GSE82853
62,E13.5 Liver,RNA-seq,GSM2071423,GSE78391


In [11]:
## merge geo id, sra id together
## generate sample name

# geo_sample_dict = {}
sample_name_dict = {}
sample_name_dict['heart'] = {}
sample_name_dict['liver'] = {}
# for tissue in ['heart', 'liver'] :
for tissue in ['heart', 'liver'] :
    for stage in data_geo_dict[tissue]['Stage'].unique() :
        df = data_geo_dict[tissue].loc[data_geo_dict[tissue]['Stage'] == stage].reset_index()
        for k,data in enumerate(df['Data'].unique()) :
            sub_df = df.loc[df['Data'] == data].reset_index()            
            for j,gsm in enumerate(sub_df['GSMID'].unique()) :
                sub_sub_df = sub_df.loc[sub_df['GSMID'] == gsm].drop('level_0', axis = 1).reset_index()
                data_type = data if data == 'WGBS' else data.split(' ')[0] if data[-8:] == "ChIP-seq" else "RNASeq"
                sample_name_dict[tissue][gsm] = f"{stage.split()[0]}.{data_type}.rep{j+1}"
        
    sample_name_df = pd.DataFrame.from_dict(sample_name_dict[tissue], orient='index').reset_index()
    sample_name_df.columns = ['GSMID', 'sample_name']
    data_geo_dict[tissue] = pd.merge(data_geo_dict[tissue], sample_name_df, on = 'GSMID')
#     geo_sra_merge_dict[tissue].to_csv(f'geo_sra_id_{tissue}.csv', sep = ",", index = False, header = True)

data_geo_dict[tissue] 

,Stage,Data,GSMID,GSEID,sample_name
0,E11.5 Liver,RNA-seq,GSM2071370,GSE78357,E11.5.RNASeq.rep1
1,E11.5 Liver,RNA-seq,GSM2071371,GSE78357,E11.5.RNASeq.rep2
2,E12.5 Liver,RNA-seq,GSM2192083,GSE82853,E12.5.RNASeq.rep1
3,E12.5 Liver,RNA-seq,GSM2192084,GSE82853,E12.5.RNASeq.rep2
4,E13.5 Liver,RNA-seq,GSM2071423,GSE78391,E13.5.RNASeq.rep1
5,E13.5 Liver,RNA-seq,GSM2071424,GSE78391,E13.5.RNASeq.rep2
6,E14.5 Liver,RNA-seq,GSM2071597,GSE78472,E14.5.RNASeq.rep1
7,E14.5 Liver,RNA-seq,GSM2071598,GSE78472,E14.5.RNASeq.rep2
8,E14.5 Liver,RNA-seq,GSM2400094,GSE90196,E14.5.RNASeq.rep3
9,E14.5 Liver,RNA-seq,GSM2400095,GSE90196,E14.5.RNASeq.rep4


In [12]:
for tissue in ['heart', 'liver'] :
    print("\n")
    for gsm in sample_name_dict[tissue].keys() :
        sample_name = sample_name_dict[tissue][gsm]
        datatype = sample_name.split('.')[-2]
        cmd5 = f'RNAseq_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/{tissue[0].upper()}{tissue[1:]}Development GEO {gsm} SINGLE {sample_name} mm10 > /mnt/Storage/home/wangyiman/CHMsInOtherContexts/bin/log/{sample_name}_mapping.log 2>&1'
        print(cmd5)




RNAseq_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment GEO GSM2343151 SINGLE E10.5.RNASeq.rep1 mm10 > /mnt/Storage/home/wangyiman/CHMsInOtherContexts/bin/log/E10.5.RNASeq.rep1_mapping.log 2>&1
RNAseq_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment GEO GSM2343152 SINGLE E10.5.RNASeq.rep2 mm10 > /mnt/Storage/home/wangyiman/CHMsInOtherContexts/bin/log/E10.5.RNASeq.rep2_mapping.log 2>&1
RNAseq_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment GEO GSM2071523 SINGLE E11.5.RNASeq.rep1 mm10 > /mnt/Storage/home/wangyiman/CHMsInOtherContexts/bin/log/E11.5.RNASeq.rep1_mapping.log 2>&1
RNAseq_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDevelopment GEO GSM2071524 SINGLE E11.5.RNASeq.rep2 mm10 > /mnt/Storage/home/wangyiman/CHMsInOtherContexts/bin/log/E11.5.RNASeq.rep2_mapping.log 2>&1
RNAseq_processing_publicData.sh ~/CHMsInOtherContexts/CellStateTransition/HeartDev